In [1]:
import pandas as pd
#import dask.dataframe as dd


In [2]:
import numpy

In [3]:
os.getcwd()

'/Users/sangersteel/Documents/jigsaw-toxic-severity-rating'

In [3]:
import tensorflow as tf


2.7.0


In [3]:
!pip install -U numpy==1.19.2


  Using cached numpy-1.19.2.zip (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  ERROR: Command errored out with exit status 1:
   command: /Users/sangersteel/miniforge3/envs/myenv/bin/python3.8 /Users/sangersteel/miniforge3/envs/myenv/lib/python3.8/site-packages/pip/_vendor/pep517/in_process/_in_process.py build_wheel /var/folders/gv/zxch4f5j72d3kxhj77v41_l00000gn/T/tmpcx_7bcm1
       cwd: /private/var/folders/gv/zxch4f5j72d3kxhj77v41_l00000gn/T/pip-install-fbo32r63/numpy_f31e9efe4ac34314bc2a5028e92eaaa7
  Complete output (884 lines):
  Running from numpy source directory.
  Cythonizing sources
  numpy/random/_bounded_integers.pxd.in has not changed
  numpy/random/_philox.pyx has not changed
  numpy/random/_bounded_integers.pyx.in has not changed
  numpy/random/_sfc64.pyx has not changed
  numpy/random/_mt19937.pyx has not changed
  numpy/random/bit_generator.pyx has not changed
  Processi

# Where I've left off:

1. Check that all trailing backslashes are out of the csv I've outputted. Use the output csv for further working

2. Following https://towardsdatascience.com/an-easy-tutorial-about-sentiment-analysis-with-deep-learning-and-keras-2bf52b9cba91 

3. Need to vectorize all the sequences in the csv and continue

In [8]:
new_dat = pd.read_csv('new_val.csv', header= None)
new_dat.columns = ['workers', 'less_toxic','more_toxic']

In [9]:
length_of_the_less_toxics = new_dat['less_toxic'].str.split(" ")
length_of_the_more_toxics = new_dat['more_toxic'].str.split(" ")


In [10]:
length_of_the_less_toxics.str.len().max()

340

In [11]:
length_of_the_more_toxics.str.len().max()

340

In [12]:
max_words = 340

In [13]:
full_corpus = []
for index, row in new_dat.iterrows():
    full_corpus.append(row['less_toxic'])
    full_corpus.append(row['more_toxic'])


In [14]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(full_corpus)

In [15]:
sequences = tokenizer.texts_to_sequences(full_corpus)

In [16]:
sequences

[[5, 118, 9055, 21404],
 [14372, 2199, 1518, 7, 529, 1, 66, 58],
 [200,
  11,
  931,
  874,
  10,
  73,
  1320,
  305,
  3673,
  64,
  8,
  949,
  239,
  2033,
  6972,
  33,
  2033,
  2035,
  2708],
 [34607,
  14373,
  609,
  21405,
  2356,
  2357,
  63,
  421,
  160,
  1820,
  579,
  34606,
  568,
  5329,
  2775,
  58,
  170,
  537,
  34608,
  1628,
  2853],
 [1004, 626, 5728, 117, 728, 412, 108, 237, 1341, 249],
 [14374,
  105,
  456,
  833,
  3144,
  1988,
  3512,
  160,
  8,
  8,
  7887,
  21,
  584,
  29,
  4,
  1166,
  1095,
  640,
  260,
  2478,
  833,
  122,
  4991,
  4449,
  10988,
  9056,
  372,
  1728,
  1351,
  729,
  865,
  41,
  77,
  6],
 [44, 14375, 140, 8, 652, 63, 7, 608, 85, 324],
 [69, 2479, 1580, 179, 22, 1884, 5729, 884, 150],
 [4450, 1580, 14376, 2036, 10989, 1352, 2553, 229, 689, 4206, 2067],
 [176, 36, 199, 2200, 249],
 [25, 285, 126, 11, 6, 3513, 17],
 [111, 9, 133, 1280, 14],
 [176,
  476,
  26,
  295,
  9,
  14377,
  2,
  13,
  712,
  2480,
  14378,
  14379,

In [17]:
tokenizer.sequences_to_texts(sequences)[:10]

['article suck woo wooooooo',
 'wher sexy pic go main page put back',
 'yes people recognize usually one first objection hear directed someone say favor free market invariably thing market ridiculously trivial',
 'daphne guinness top mornin favourite alien life form sorry marry poor commoner therefore unimportant dress back hope fat oirish arse stretch',
 'western medium yup every crime entire world cause west racist',
 'atom believe actual photo masturbation sexually explicit sorry say say disclaimer use purpose also wikipedia target minor addition child pornography photo real prepubescent nude illustration bondage title virgin killer recently archive comment could make',
 'remove numbskull care say anymore life go ahead leave alone',
 'seem sand vagina might want wash stinking hole bitch',
 'smelly vagina bluerasberry model cheesy blue syndrome lot experience vaginal disease',
 'hey way support nazis racist']

In [18]:
from keras.preprocessing.sequence import pad_sequences

paddings = pad_sequences(sequences)

In [19]:
import numpy as np 

np.shape(paddings)

(60216, 340)

In [20]:
ser = pd.Series(sequences)

number_of_possible_tokens = ser.apply(lambda x: max(x)).max()

In [21]:
dimensionality_of_vecspace = np.shape(paddings)[1]

In [22]:
from keras.layers import Embedding
embedding_layer = Embedding(number_of_possible_tokens, dimensionality_of_vecspace)

In [23]:
dimensionality_of_vecspace

340

In [24]:
max_words

340

In [25]:
import numpy
numpy.__version__

'1.19.5'

In [146]:
len_x = len(new_dat.index)

In [154]:
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint

model1 = Sequential()
model1.add(layers.Embedding(max_words, len_x)) #The embedding layer
model1.add(layers.LSTM(units = dimensionality_of_vecspace, dropout = 0.5))
model1.add(layers.Dense(2,activation='softmax'))


model1.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)


# Now need x_train and y_train


My proposed way of dealing with this is splitting x_data in tuples, of the two comments, and then outputting for y_train a tuple of like (a,b) where a or b is 1, and the other is 0, indicating which one is more toxic. I'm worried this'll just make all y_train (0,1), which might make the model just take the second option by default. Can maybe manage this by just scrambling the order of the comments? I don't know -- sounds like cheating. 

In [125]:
# Group up every 2. Odds are less toxics, evens are more toxics

collections = len(np.arange(0,len(sequences),2))

x_train = []
y_train = []
for i in np.arange(0,len(paddings),2):
    #print(i)
    determiner = np.random.randint(0,2)
    if determiner == 0:
        x_train.append([paddings[i], paddings[i+1]])
        y_train.append([0,1])
    elif determiner == 1:
        x_train.append([paddings[i+1], paddings[i]])
        y_train.append([1,0])
    

In [133]:
paddings[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [129]:
x_train = tf.convert_to_tensor(x_train)
y_train = tf.convert_to_tensor(y_train)

In [158]:
x_train[0]

<tf.Tensor: shape=(2, 340), dtype=int32, numpy=
array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [155]:
model1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 30108)       10236720  
                                                                 
 lstm_1 (LSTM)               (None, 340)               41410640  
                                                                 
 dense_1 (Dense)             (None, 2)                 682       
                                                                 
Total params: 51,648,042
Trainable params: 51,648,042
Non-trainable params: 0
_________________________________________________________________


In [156]:
history = model1.fit(x_train, y_train, epochs=70,callbacks=[checkpoint1])

Epoch 1/70


ValueError: in user code:

    File "/Users/sangersteel/miniforge3/envs/toxic/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/Users/sangersteel/miniforge3/envs/toxic/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/sangersteel/miniforge3/envs/toxic/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/Users/sangersteel/miniforge3/envs/toxic/lib/python3.8/site-packages/keras/engine/training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "/Users/sangersteel/miniforge3/envs/toxic/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/sangersteel/miniforge3/envs/to